In [2]:
# Data preprocessor For Neural Question Generator
import os
import json
import pandas as pd
import re

In [2]:
with open('data.json') as f:
    data = json.load(f)

In [3]:
# https://github.com/christopherbate/nlp-squad-preprocessing/blob/master/squad_processer.py reference
# I made many alterations to their code so it could match our specific needs. They mostly gave me an idea
# of how the data was set up
questions = []
answers = []
contexts = []
for i in range(len(data['data'])):
    temp = data['data'][i]['paragraphs']
    for j in range(len(temp)):
        context = temp[j]['context']
        context = re.split(r"[.!?]",context)
        context = [item.strip(" ") for item in context]
        if len(context) > 2:
            context = context[:-1]
        qas = temp[j]['qas']
        for k in range(len(qas)):
            question = qas[k]['question']
            value = 0
            answer = qas[k]['answers']
            if not answer:
                answer = qas[k]['plausible_answers'][0]['text']
            else:
                answer = answer[0]['text']  
            question = re.sub("[\(\[].*?[\)\]]", "", question)
            answer = re.sub("[\(\[].*?[\)\]]", "", answer)
            #If the answer is in a sentence, when save up the sentence question pair
            for item in context:
                if answer in item:
                    context = re.sub("[\(\[].*?[\)\]]", "",item)
                    questions.append(re.sub('\s+',' ',question))
                    answers.append(re.sub('\s+',' ',answer))
                    contexts.append(re.sub('\s+',' ',context))
                    break
                    
            

In [6]:
#Save dataframe
dat = pd.DataFrame(answers)
dat['question'] = questions
dat.to_csv('neuralquestion.txt',sep='\t',header=False,index=False)

In [22]:
# Now This will preprocess the Trump data
tweets = pd.read_csv('trumpdata')

In [23]:
#Preprocess: Elimiate all useless stuff and just grab tweets with certain keywords
data = tweets[tweets.text.str.contains("Russia|Republican|Fake|News|Hillary|MAGA|CNN|President|War|USA|money|win|Democrats|America|huge|best|win|money") == True]
data = [re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",item) for item in data.iloc[:,0]]
data = [" ".join(item.split()) for item in data]

In [24]:
#Save to right format
data = pd.DataFrame(data)
data.to_csv('TrumpData.txt',sep='\t',header=False,index=False)